<a href="https://colab.research.google.com/github/Rufaidah44/Hona_Malam/blob/main/Hona_Malam_to_run_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat Bot


In [11]:
# libraries
import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


# chat initialization
model = load_model("chatbot_model.h5")
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

app = Flask(__name__, template_folder='drive/MyDrive/Colab Notebooks/web_chatbot/templates')
run_with_ngrok(app) 

@app.route("/")
def home():
    return render_template("index.html")


OSError: ignored

In [ ]:
@app.route("/get", methods=["POST"])
def chatbot_response():
    msg = request.form["msg"]
    if msg.startswith('my name is'):
        name = msg[11:]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
    elif msg.startswith('hi my name is'):
        name = msg[14:]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
    else:
        ints = predict_class(msg, model)
        res = getResponse(ints, intents)
    return res

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result



if __name__ == "__main__":
    app.run()

# Classification Images


In [ ]:
!pip install flask_ngrok
!pip install flask_uploads
!pip install pip install werkzeug==0.16.0

In [ ]:
from werkzeug.utils import secure_filename
from werkzeug.datastructures import  FileStorage
from flask_uploads import UploadSet
from flask_uploads import configure_uploads
from flask_uploads import IMAGES
from keras.preprocessing.image import load_img
from keras.preprocessing import image
from flask_ngrok import run_with_ngrok
import flask
from flask import Flask, render_template, request
from keras.models import load_model
import numpy as np

model = load_model("/content/image_my_h5_model.h5")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

pred=model.predict(validation_generator,
steps=validation_generator.samples // BATCH_SIZE,
verbose=1)

In [ ]:
model = load_model("/content/image_my_h5_model.h5")


app = flask.Flask(__name__, template_folder='drive/MyDrive/Colab Notebooks/web_chatbot/templates')
run_with_ngrok(app)
#app = Flask(__name__)

photos = UploadSet('photos', IMAGES)

# path for saving uploaded images
app.config['UPLOADED_PHOTOS_DEST'] = './static/img'
configure_uploads(app, photos)

# professionals have standards :p
#@app.route('/home', methods=['GET', 'POST'])
#def home():
#   welcome = "Hello, World !"
#   return welcome

# the main route for upload and prediction
@app.route('/', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST' and 'photo' in request.files:
        # save the image
        filename = photos.save(request.files['photo'])
        # load the image
        path='./static/img/'+filename
        img = image.load_img(path, target_size=(224, 224))

        x = image.img_to_array(img)
        img1 = np.array(x).astype('float32')/255
        #img2 = transform.resize(img1, (224, 224))
        #x=np.expand_dims(x, axis=0)
        img = np.expand_dims(img1, axis=0)
        img = np.vstack([img])
        r = model.predict(img)

        names= dict((v,k) for k,v in labels.items())
        index = np.argmax(r)
        name=names.get(index)

        p=round(r.max()*100, 3)  #to find maximum score
        names= dict((v,k) for k,v in labels.items())
        value = np.argmax(r)

        # list out keys and values separately
        key_list = list(names.keys())
        val_list = list(names.values())
        # print key with val 100
        position = val_list.index(value)
        answer = " :صورة المعلم المدخله هي لـ"+ str(key_list[position])
        #plt.imshow(img1)
        return answer


    # web page to show before the POST request containing the image
    return render_template('upload.html')


if __name__ == '__main__':
    app.run()